# 第十二篇：样式

Pandas 的 Style 是一个正在被大多数人忽视的可视化方法，它不仅仅是为了美化一下数据提高数据的颜值这么简单。回忆一下，或许你刚刚这么做过，我们在 Excel 中是不是经常对特定的数据加粗、标红、背景黄，又红又粗加背景黄。这些操作就是让我们看数据更加醒目清晰，突显出数据的逻辑和特征来。
样式的基本理念是用户在调整数据显示形式的同时，不会影响数据本身的计算操作。
最直观的样式是货币符号，如果只显示一个数字，比如 25.06，读者并不清楚这个数字代表的到底是美元、英镑、日元还是人民币，但 ￥25.06，大家就都知道这是人民币了，由此可见，样式可以让数据提供更多信息。
百分比也是数据样式的重要形式，0.12 就不如 12% 更直观，用百分比说明数据更清晰，也更简单。

Pandas 支持不同样式，可以用更易理解的方式显示数据，但并未修改数据的类型，所以依然还可以用 Pandas 提供的各种数学、日期、字符串函数处理数据。

Pandas 样式还包括内容里提及的高级样式，比如添加颜色与条形图等可视化元素。

**功能**

主要的功能有：

- 数值格式化，如千分号、小数位数、货币符号、日期格式，百分比等等
- 突显某些数据，对行、列或者特定的值（如最大最小值）使用样式，如字体大小、黄色、背景
- 显示数据关系，如用颜色深浅代表数据大大小
- 迷你条形图，如在一个百分比的格子里，用颜色比例表达占比
- 表达趋势，类似 Excel 中每行代表趋势变化的迷你走势图（sparkline）
- 等等。

我们发现，样式和可视化图表的区别是，图表一般数据图形化不关注具体数据内容，而样式则在保留具体内容的基础上进行修饰，让可读性更强。有时候两者有交叉共用的情况。

**Styler 对象**

DataFrame 有一个 `df.style` Styler 对象，用来生成数据的样式，样式是使用 CSS 来完成的，如果你懂点 CSS 的知识会得心应手很多，不过也不用担心，它非常简单，基本就是一个字典，英文也是我们最常见的。

这儿有个技巧，使用 style 可以在 notebook 上未给样式的情况下显示所有的数据：

```python
df.style # 可实现显示所有数据
```

所有的样式功能都在 `df.style` 里。需要注意的是，输出的是一个 Styler 对象不是 DataFrame，原始 DataFrame 内容并没有改变。

**样式的导出**

带有样式的 `df.style` 可以使用 `df.style.to_excel()` 导出，导出后的 excel 会保留样式。

In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_excel('data/team.xlsx')
df.head()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,C,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


## 第一部分：内置样式
Pandas 提供了几个非常常用的内置样式，是我们经常要使用的，这一部分将对这些提高计数效率的样式功能进行介绍。

### 空值高亮 highlight_null
对为空的值，高亮标示。

In [3]:
df.iloc[1,1] = np.NaN # 修改一个值空
df.head().style.highlight_null() # 默认红色

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


可以指定颜色：

In [4]:
# 使用颜色名
df.head().style.highlight_null(null_color='blue')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [5]:
# 使用颜色值
df.head().style.highlight_null(null_color='#ccc')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


### 最大最小值高亮

In [6]:
# 最大值高亮
df.head().style.highlight_max()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [7]:
# 最小值高亮
df.head().style.highlight_min()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [8]:
# 同时使用+指定颜色
(df.head()
 .style.highlight_max(color='lime') # 最大值高亮
 .highlight_min() # 最小值
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


以上在在一列中的最大和最小，也可以指定在行上的最大和最小：

In [9]:
# 指定行级
(df.head()
 .style.highlight_max(color='lime', axis=1) # 最大值高亮, 绿色
 .highlight_min(axis=1) # 最小值，黄色
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


也可以作用于指定行：

In [10]:
# 只对 数量 起作用
df.head().style.highlight_min(subset=['Q1'])

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [11]:
df.head().style.highlight_min(subset=['Q1', 'Q2']) # 两列

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [12]:
# 使用 pd.IndexSlice 索引器（和 loc[] 类似）
# 注意，数据是前10行，算最小值的范围是前5行
df[:10].style.highlight_min(subset=pd.IndexSlice[:4, ['Q1', 'Q2']])

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [13]:
# 按行，只在这四列进行
df[:10].style.highlight_min(axis=1, subset=['Q1','Q2', 'Q3', 'Q4'])

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 背景渐变 background_gradient
根据数值的大小背景颜色呈现梯度渐变，越深表示越大，越浅表示越小，类似于 Excel 的中的色阶样式。颜色指定为 Matplotlib 库的色系表（Matplotlib colormap）中的色系名, 点击查看所有色表。

In [14]:
# 数字类型按列背景渐变
df[:10].style.background_gradient()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [15]:
# 低百分比和高百分比范围, 更换颜色时避免使用所有色域
df[:10].style.background_gradient(low=0.6, high=0)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [16]:
# 内容的颜色，取 0-1（深色到浅色），方便突显出文本
df[:10].style.background_gradient(text_color_threshold=0.5)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [17]:
# 颜色应用的取值范围，不在这个范围的不应用
df[:10].style.background_gradient(vmin=60, vmax=100)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


以下是一个综合使用示例：

In [18]:
# 链式方法使用样式
(df.head(15)
 .style
 .background_gradient(subset=['Q1'], cmap='spring') # 指定色系
 .background_gradient(subset=['Q2'], vmin=60, vmax=100) # 指定应用值区间
 .background_gradient(subset=['Q3'], low=0.6, high=0) # 高低百分比范围
 .background_gradient(subset=['Q4'], text_color_threshold=0.9) # 文本色深
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 条形图 bar
条形图在表格里一般以横向柱状图的形式代表这个值的大小。

In [19]:
# 基本使用，默认将数字应用
df[:10].style.bar()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [20]:
# 指定应用范围
df[:10].style.bar(subset=['Q1', 'Q2'])

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [21]:
# 定义颜色
df[:10].style.bar(color='green')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [22]:
df[:10].style.bar(color='#ff11bb')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [23]:
# 横向进行计算展示
df[:10].style.bar(axis=1)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [24]:
# 样式在格中的占位百分比，0-100, 100占满
df[:10].style.bar(width=80)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [25]:
# 对齐方式：
# ‘left’ 最小值开始,
# ‘zero’ 0值在中间,
# ’mid’  (max-min)/2 值在中间，负（正）值0在右（左）
df[:10].style.bar(align='mid')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [26]:
# 大小基准值
df[:10].style.bar(vmin=60, vmax=100)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


以下是一个综合示例：

In [27]:
(df.head(10)
 .assign(avg=df.mean(axis=1, numeric_only=True)) # 增加平均值
 .assign(diff=lambda x: x.avg.diff()) # 和前位同学差值
 .style
 .bar(color='yellow', subset=['Q1'])
 .bar(subset=['avg'],
      width=90,
      align='mid',
      vmin=60, vmax=100,
      color='#5CADAD')
 .bar(subset=['diff'],
      color=['#ffe4e4','#bbf9ce'], # 上涨下降的颜色
      vmin=0, vmax=30, # 范围定以0为基准的上下30
      align='zero') # 0 值居中
)

,name,team,Q1,Q2,Q3,Q4,avg,diff
0,Liver,E,89,21,24,64,49.500000,nan
1,Arry,nan,36,37,37,57,41.750000,-7.750000
2,Ack,A,57,60,18,84,54.750000,13.000000
3,Eorge,C,93,96,71,78,84.500000,29.750000
4,Oah,D,65,49,61,86,65.250000,-19.250000
5,Harlie,C,24,13,87,43,41.750000,-23.500000
6,Acob,B,61,95,94,8,64.500000,22.750000
7,Lfie,A,9,10,99,37,38.750000,-25.750000
8,Reddie,D,64,93,57,72,71.500000,32.750000
9,Oscar,A,77,9,26,67,44.750000,-26.750000


## 第二部分：格式显示
我们在最终输出数据查看时，需要对数据进行相应的格式化，常见的如加货币符号、加百分号、增加千分位等，目的是让计数更加场景化，明确列表一定的业务意义。

Styler.format 是专门用来处理格式的方法。

### 语法结构
```python
Styler.format(self, formatter,
              subset=None,
              na_rep: Union[str, NoneType]=None)
```
formatter （str, callable, dict or None）一般是一个字典，由列名和格式组成，也可以给一个函数来调用。

关于字符的格式化，可参考 python 教程中的格式化字符串。

### 用法
```python
# 百分号，类似 29.57%
df[:10].style.format("{:.2%}")
# 指定列全变为大写
df.style.format({'name': str.upper})
# B 保留四位，D 两位小数并显示正负号
df.style.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})
# 应用 lambda
df.style.format({"B": lambda x: "±{:.2f}".format(abs(x))})
# 缺失值的显示格式
df.style.format("{:.2%}", na_rep="-")
# 处理内置样式函数的缺失值
df.style.highlight_max().format(None, na_rep="-")
# 常用的格式
{'a': '¥{0:,.0f}', # 货币符号
 'b': '{:%Y-%m}', # 年月
 'c': '{:.2%}',  # 百分号
 'd': '{:,f}',  # 千分位
 'e': str.upper} # 大写
```

### 综合案例

In [28]:
# 链式方法使用格式
(df.head(15)
 .head(10)
 .assign(avg=df.mean(axis=1, numeric_only=True)/100) # 增加平均值百分比
 .assign(diff=lambda x: x.avg.diff()) # 和前位同学差值
 .style
 .format({'name': str.upper})
 .format({'avg': "{:.2%}"})
 .format({'diff': "¥{:.2f}"}, na_rep="-")
)

,name,team,Q1,Q2,Q3,Q4,avg,diff
0,LIVER,E,89,21,24,64,49.50%,-
1,ARRY,nan,36,37,37,57,41.75%,¥-0.08
2,ACK,A,57,60,18,84,54.75%,¥0.13
3,EORGE,C,93,96,71,78,84.50%,¥0.30
4,OAH,D,65,49,61,86,65.25%,¥-0.19
5,HARLIE,C,24,13,87,43,41.75%,¥-0.23
6,ACOB,B,61,95,94,8,64.50%,¥0.23
7,LFIE,A,9,10,99,37,38.75%,¥-0.26
8,REDDIE,D,64,93,57,72,71.50%,¥0.33
9,OSCAR,A,77,9,26,67,44.75%,¥-0.27


## 第三部分：样式配置

Pandas 对样式可以做一些整体性的配置，保证后继无需逐一去进行设置。同时，还提供了一些操作方法，使样式内容的输出更新丰富。

### 常用操作
标题 caption
给表格一个标题：

In [29]:
df.head().style.set_caption('学生成绩表')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


### 精度 Precision
可以设置全局的数据精度，即保留小数的位数。

In [30]:
# 保留两个小数
df[:10].style.set_precision(2)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [31]:
# 等同于
df[:10].round(2).style

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 缺失值 Missing values
缺失值的统一设置。

In [32]:
df[:10].style.set_na_rep("暂无")

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,暂无,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 隐藏索引和列

In [33]:
# 不输出索引
df.head().style.hide_index()

name,team,Q1,Q2,Q3,Q4
Liver,E,89,21,24,64
Arry,nan,36,37,37,57
Ack,A,57,60,18,84
Eorge,C,93,96,71,78
Oah,D,65,49,61,86


In [34]:
# 不输出指定列
df[:10].style.hide_columns(['name','team'])

,Q1,Q2,Q3,Q4
0,89,21,24,64
1,36,37,37,57
2,57,60,18,84
3,93,96,71,78
4,65,49,61,86
5,24,13,87,43
6,61,95,94,8
7,9,10,99,37
8,64,93,57,72
9,77,9,26,67


### 表格样式 Table styles
设置内联样式 set_properties
给单元格设置 CSS 样式。

In [35]:
# 指定列，设置字色为红色
df[:10].style.set_properties(subset=['Q1'], **{'color': 'red'})

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


一些其他示例

In [36]:
df.head().style.set_properties(color="white", align="right")

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [37]:
df.head().style.set_properties(**{'background-color': 'yellow'})

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [38]:
df.head().style.set_properties(**{'width': '100px', 'font-size': '18px'})

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [39]:
df.head().style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


### 表格属性 set_table_attributes
给 `<table> `标签增加属性，可以随意给定属性名和属性值。

In [40]:
df.head().style.set_table_attributes('class="pure-table"')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [41]:
df.head().style.set_table_attributes('id="gairuo-table"')

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


### 表格样式属性指定 set_table_styles
每个单独的table_样式都应该是一个带有选择器和props键的字典。选择器应该是样式将应用于的CSS选择器（自动以表的UUID作为前缀），props应该是具有（属性、值）的元组列表。

In [42]:
# 给所有的行（tr 标签）的 hover 方法给一个黄色背景
# 人话说是鼠标移动上去整行背景变黄
df.head().style.set_table_styles(
    [{'selector': 'tr:hover',
      'props': [('background-color', 'yellow')]}]
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [43]:
# 按列设置不同的属性 overwrite 可设置替换还是扩展 pd 1.2.0+
df[:10].style.set_table_styles({
    'Q1': [{'selector': '',
           'props': [('color', 'red')]}],
    'Q2': [{'selector': 'td',
           'props': [('color', 'blue')]}]
}, overwrite=False)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [44]:
# axis 可按列操作 pd 1.2.0+
df.head().style.set_table_styles({
    0: [{'selector': 'td:hover',
         'props': [('font-size', '25px')]}]
}, axis=1, overwrite=False)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


### 单元格符缀 set_uuid

In [45]:
# 给每个表格给一个相同的符缀
df.head().style.set_uuid(9999)
# ... <td id="T_9999row0_col2"  ..

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [46]:
# 加 gairuo
df.head().style.set_uuid('gairuo')
# ... <td id="T_gairuorow0_col2" ...

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


## 第四部分：样式应用函数
像 Series 或者 DataFrame 的 apply() 和 applymap() 一样，Styler 也可以使用它们进行样式的复杂定义。

### 应用函数
样式调用函数的方法有：

- Styler.applymap: 所有元素
- Styler.apply: 列/行/表

这两个方法都接受一个函数（以及一些其他关键字参数）并以某种方式将函数应用于数据帧。applymap 在 DataFrame 所有元素中有效。apply 根据 axis 关键字参数，一次将每个列或行传递到数据框中，或一次传递到整个表中。行使用 axis=0 （默认），列使用 axis=1，对于整个表，使用 axis=None。

#### 最大值字体置为红色

In [47]:
# 最大值显示红色
def highlight_max(x):
    return ['color: red' if v == x.max() else '' for v in x]

# 应用函数
df.style.apply(highlight_max)
# 按行应用
df[:10].loc[:,'Q1':'Q4'].style.apply(highlight_max, axis=1)

,Q1,Q2,Q3,Q4
0,89,21,24,64
1,36,37,37,57
2,57,60,18,84
3,93,96,71,78
4,65,49,61,86
5,24,13,87,43
6,61,95,94,8
7,9,10,99,37
8,64,93,57,72
9,77,9,26,67


#### 以下所有大于80分的格子背景为黄色

In [48]:
yellow_css = 'background-color: yellow'
sfun = lambda x: yellow_css if type(x) == int and x > 80 else ''
(df[:10].style
 .applymap(sfun)
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


#### subset 参数可以指定部分内容应用样式规则

In [49]:
(df[:10].style
 .applymap(sfun, subset=['Q1', 'Q2'])
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


#### 对满足条件的行整行加背景样式：

In [50]:
yellow_css = 'background-color: yellow'
sfun = lambda x: [yellow_css]*len(x) if x.Q1 > 80 else ['']*len(x)
(df[:10].style
 .apply(sfun, axis=1)
)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


In [51]:
# 使用函数，字体颜色整行为红
def row_color(s):
    if s.Q1 > 80:
        return ['color: red']*len(s)
    else:
        return ['']*len(s)

# 调用
df[:10].style.apply(row_color, axis=1)

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 管道方法 Styler.pipe

In [52]:
# 定义样式函数
def format_conversion(styler):
    return (styler.set_properties(**{'text-align': 'right'})
                  .format({'conversion': '{:.1%}'}))

df1 = pd.DataFrame({'trial': list(range(5)),
                   'conversion': [0.75, 0.85, np.nan, 0.7, 0.72]})

# 使用管道方法调用
(df1[:10].style
   .highlight_min(subset=['conversion'], color='yellow')
   .pipe(format_conversion)
   .set_caption("Results with minimum conversion highlighted.")
)

,trial,conversion
0,0,75.0%
1,1,85.0%
2,2,nan%
3,3,70.0%
4,4,72.0%


### 样式复用
可以将其他的样式利用到新的表格中：

In [53]:
df2 = df.copy()
df2.head()

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,NaN,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86


In [54]:
red_css = 'background-color: red'
def color_negative_red(x):
    return red_css if type(x) == int and x < 60 else ''

sfun = lambda x: red_css if type(x) == int and x < 60 else ''
# 将 df 的样式赋值给变量
style1 = df.style.applymap(color_negative_red)
# df2 的样式为 style2
style2 = df2[:10].style
# style2 使用 style1的样式
style2.use(style1.export())

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 样式清除

In [55]:
df.style.clear() # 会返回 None，清除所有样式。

In [56]:
# 定义为一个变量
dfs = df.loc[:10,'Q1':'Q4'].style.apply(highlight_max)
dfs.clear() # 清除
dfs # 此时 dfs 不带任何样式，但还是 Styler 对象

,Q1,Q2,Q3,Q4
0,89,21,24,64
1,36,37,37,57
2,57,60,18,84
3,93,96,71,78
4,65,49,61,86
5,24,13,87,43
6,61,95,94,8
7,9,10,99,37
8,64,93,57,72
9,77,9,26,67


## 第五部分：带样式文件的导入
可以将样式生成 Html 和 导出 Excel。生成 Html 可以用它来发邮件，做网页界面，生成 Excel 可以做二次处理或者传播。

### 导出 Excel
样式导出 Excel 后会保留原来定义的样式。
```python
# 导出 Excel
df.style.to_excel('new_team.xlsx')
# 使用指定引擎，openpyxl 的样式兼容更好些
df.style.to_excel('new_team.xlsx', engine='openpyxl')
# 指定标签页名称，sheet name
dfs.to_excel('new_team.xlsx', sheet_name='Sheet1')
# 缺失值的指定
dfs.to_excel('new_team.xlsx', na_rep='-')
# 浮点数字格式, 下例将 0.1234 转为 0.12
dfs.to_excel('new_team.xlsx', float_format="%.2f")
# 只要这两例
dfs.to_excel('new_team.xlsx', columns=['Q1', 'Q2'])
# 不带表头
dfs.to_excel('new_team.xlsx', header=False)
# 不带索引
dfs.to_excel('new_team.xlsx', index=False)
# 指定索引，多个为多层索引
dfs.to_excel('new_team.xlsx', index_label=['team', 'name'])
# 从哪行取，从哪列取
dfs.to_excel('new_team.xlsx', startrow=10, startcol=3)
# 不合并单元格
dfs.to_excel('new_team.xlsx', merge_cells=False)
# 指定编码
dfs.to_excel('new_team.xlsx', encoding='utf-8')
# 无穷大表示法（Excel中没有无穷大的本机表示法）
dfs.to_excel('new_team.xlsx', inf_rep='inf')
# 在错误日志中显示更多信息
dfs.to_excel('new_team.xlsx', verbose=True)
# 指定要冻结的最底行和最右列
dfs.to_excel('new_team.xlsx', freeze_panes=(0,2))
```

### 输出 Html
Styler.render() 可以输出样式的 Html 代码，它可以传入以下参数：

- head
- cellstyle
- body
- uuid
- precision
- table_styles
- caption
- table_attributes

In [57]:
df.style.render()

'<style  type="text/css" >\n</style><table id="T_6c2ae_" ><thead>    <tr>        <th class="blank level0" ></th>        <th class="col_heading level0 col0" >name</th>        <th class="col_heading level0 col1" >team</th>        <th class="col_heading level0 col2" >Q1</th>        <th class="col_heading level0 col3" >Q2</th>        <th class="col_heading level0 col4" >Q3</th>        <th class="col_heading level0 col5" >Q4</th>    </tr></thead><tbody>\n                <tr>\n                        <th id="T_6c2ae_level0_row0" class="row_heading level0 row0" >0</th>\n                        <td id="T_6c2ae_row0_col0" class="data row0 col0" >Liver</td>\n                        <td id="T_6c2ae_row0_col1" class="data row0 col1" >E</td>\n                        <td id="T_6c2ae_row0_col2" class="data row0 col2" >89</td>\n                        <td id="T_6c2ae_row0_col3" class="data row0 col3" >21</td>\n                        <td id="T_6c2ae_row0_col4" class="data row0 col4" >24</td>\n        

In [58]:
# 过虑换行取部分，增加可读性
df.style.highlight_null().render().split('\n')[:10]

['<style  type="text/css" >',
 '#T_6be29_row1_col1{',
 '            background-color:  red;',
 '        }</style><table id="T_6be29_" ><thead>    <tr>        <th class="blank level0" ></th>        <th class="col_heading level0 col0" >name</th>        <th class="col_heading level0 col1" >team</th>        <th class="col_heading level0 col2" >Q1</th>        <th class="col_heading level0 col3" >Q2</th>        <th class="col_heading level0 col4" >Q3</th>        <th class="col_heading level0 col5" >Q4</th>    </tr></thead><tbody>',
 '                <tr>',
 '                        <th id="T_6be29_level0_row0" class="row_heading level0 row0" >0</th>',
 '                        <td id="T_6be29_row0_col0" class="data row0 col0" >Liver</td>',
 '                        <td id="T_6be29_row0_col1" class="data row0 col1" >E</td>',
 '                        <td id="T_6be29_row0_col2" class="data row0 col2" >89</td>',
 '                        <td id="T_6be29_row0_col3" class="data row0 col3" >21</td

In [59]:
# 在 notebook 中可以会用来解析展示生成的 html
from IPython.display import HTML
HTML(df[:10].style.render())

,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,nan,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
5,Harlie,C,24,13,87,43
6,Acob,B,61,95,94,8
7,Lfie,A,9,10,99,37
8,Reddie,D,64,93,57,72
9,Oscar,A,77,9,26,67


### 生成 html 的属性命名
以下是生成的 html 中各属性的命名规则。
https://pandas.pydata.org/docs/user_guide/style.html#CSS-classes